In [1]:
import os
import sys
import pandas as pd
import plotly.express as px
from google.cloud import bigquery

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")
# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [2]:
if not os.getenv("IS_TESTING"):
  # automatically restart kernel after installs
  import IPython

  app = IPython.Application.instance()
  app.kernel.do_shutdown(True)

In [2]:
PROJECT_ID = ""
if PROJECT_ID == "" or PROJECT_ID is None:
  PROJECT_ID = "flowing-indexer-385101"

In [3]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS '

In [4]:
#Client manages connections to the BigQuery API and helps
#bundle configuration (project, credentials) needed for API requests.
client = bigquery.Client(PROJECT_ID)

# to make sure all columns are displayed while working with dataframe
pd.set_option('display.max_columns', None)

In [5]:
PROJECT_ID_DATA = "bigquery-public-data"
DATASET_ID_DATA = "ga4_obfuscated_sample_ecommerce"
START_DATE = "20201101"
END_DATE = "20210131"

In [6]:
query = f"""
SELECT 
*
FROM 
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events*`
LIMIT
  5
"""
print(query)
query_job = client.query(query)
top5_data = query_job.to_dataframe()
top5_data.head()


SELECT 
*
FROM 
  `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events*`
LIMIT
  5



,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,user_pseudo_id,privacy_info,user_properties,user_first_touch_timestamp,user_ltv,device,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items
0,20201224,1608798440121768,user_engagement,"[{'key': 'debug_mode', 'value': {'string_value...",<NA>,NaN,-4091929387,<NA>,None,1013488.1622938896,"{'analytics_storage': None, 'ads_storage': Non...",[],1608798235448219,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Europe', 'sub_continent': 'Sout...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
1,20201224,1608798560911047,user_engagement,"[{'key': 'all_data', 'value': {'string_value':...",<NA>,NaN,-1721763749,<NA>,None,1013488.1622938896,"{'analytics_storage': None, 'ads_storage': Non...",[],1608798235448219,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Europe', 'sub_continent': 'Sout...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
2,20201224,1608798905421149,scroll,"[{'key': 'percent_scrolled', 'value': {'string...",<NA>,NaN,-6825949689,<NA>,None,1013488.1622938896,"{'analytics_storage': None, 'ads_storage': Non...",[],1608798235448219,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Europe', 'sub_continent': 'Sout...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
3,20201224,1608798567030686,view_search_results,"[{'key': 'search_term', 'value': {'string_valu...",<NA>,NaN,-8462447293,<NA>,None,1013488.1622938896,"{'analytics_storage': None, 'ads_storage': Non...",[],1608798235448219,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Europe', 'sub_continent': 'Sout...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]
4,20201224,1608798235448219,first_visit,"[{'key': 'ga_session_id', 'value': {'string_va...",<NA>,NaN,-6852730992,<NA>,None,1013488.1622938896,"{'analytics_storage': None, 'ads_storage': Non...",[],1608798235448219,"{'revenue': 0.0, 'currency': 'USD'}","{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Europe', 'sub_continent': 'Sout...",None,"{'medium': 'organic', 'name': '(organic)', 'so...",2100450278,WEB,None,"{'total_item_quantity': None, 'purchase_revenu...",[]


In [7]:
query = f"""
SELECT 
  DISTINCT(column_name),
  data_type
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.INFORMATION_SCHEMA.COLUMNS`
"""

query_job = client.query(query)
data = query_job.to_dataframe()
data

,column_name,data_type
0,event_date,STRING
1,event_timestamp,INT64
2,event_name,STRING
3,event_params,"ARRAY<STRUCT<key STRING, value STRUCT<string_v..."
4,event_previous_timestamp,INT64
5,event_value_in_usd,FLOAT64
6,event_bundle_sequence_id,INT64
7,event_server_timestamp_offset,INT64
8,user_id,STRING
9,user_pseudo_id,STRING


In [8]:
query = f"""
SELECT
  COUNT(DISTINCT event_name) AS event_count,
  COUNT(DISTINCT user_pseudo_id) AS user_count,
  COUNT(DISTINCT event_date) AS day_count,
  COUNT(DISTINCT user_id) AS registered_user_id
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events*`

"""

query_job = client.query(query)
top5_data = query_job.to_dataframe()
top5_data

,event_count,user_count,day_count,registered_user_id
0,17,270154,92,0


In [9]:
query = f"""
SELECT 
  event_name,
  COUNT(*) as row_count
FROM
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events*`
  GROUP BY 1
  ORDER BY 2 DESC
"""

query_job = client.query(query)
result_df = query_job.to_dataframe()
fig = px.bar(result_df, x= "row_count",y = "event_name", title = "Event Name Frequency Distribution")
fig.show()

In [10]:
query = f"""
  SELECT
    DISTINCT(ep.key) AS event_param_key,
    COUNT(*) AS cnt
  FROM
    `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events*`,
    UNNEST (event_params) AS ep
  WHERE
    event_name = 'page_view'
  GROUP BY 1
  ORDER BY 2 DESC
  """
query_job = client.query(query)
result_df = query_job.to_dataframe()

fig = px.bar(result_df.head(20), x="cnt", y = "event_param_key", title = "Event Parameters in Various Events Frequency Distribution")
fig.show()

In [11]:
query = """
  SELECT 
  DISTINCT(ep.value.string_value) AS page_title,
  COUNT(*) AS page_title_count
  FROM 
  `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`,
  UNNEST (event_params) as ep
  WHERE
  ep.key = 'page_title'
  AND event_name = 'page_view'
  GROUP BY 1
  ORDER BY 2 DESC
  LIMIT 20
  """
query_job = client.query(query)
page_title_result_df = query_job.to_dataframe()

fig = px.bar(page_title_result_df,x = "page_title_count", y = "page_title", title = "Different Page Visited Frequency Distribution")
fig.show()


I0429 14:04:28.254730 140057357485888 bigquery_client.py:730] There is no apilog flag so non-critical logging is disabled.
Invalid identifier 'ga_ecomm_feature_set' for mk.


In [14]:
# Set the name for your new dataset
dataset_name = "ga4_ecomm_feature_set"

# Construct a reference to the new dataset
dataset_ref = client.dataset(dataset_name)

# Set the location for the new dataset
# location = "US"

# Create the new dataset
dataset = bigquery.Dataset(dataset_ref)
#dataset.location = location
dataset = client.create_dataset(dataset)

print(f"Created dataset {dataset.dataset_id}")

Created dataset ga4_ecomm_feature_set


In [33]:
exclude_list = ('Home','Google Online Store')
top_n = 20

query = f"""
WITH
  top_viewed_pages AS (
    SELECT
    DISTINCT(ep.value.string_value) AS page_title,
    REGEXP_REPLACE(REPLACE(TRIM(SPLIT(ep.value.string_value, "|")[SAFE_OFFSET(0)]), " ", "_"), r'[^a-zA-Z]','') AS key,
    COUNT(*) AS page_title_count
    FROM
    `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events_*`,
    UNNEST (event_params) AS ep
    WHERE
    ep.key = 'page_title'
    GROUP BY 1
    ORDER BY 3 DESC
    LIMIT {top_n}
  )
  SELECT *
  FROM top_viewed_pages
  WHERE page_title NOT IN {exclude_list}
  """

query_job = client.query(query)
df = query_job.to_dataframe()
df

,page_title,key,page_title_count
0,Apparel | Google Merchandise Store,Apparel,235947
1,Shopping Cart,ShoppingCart,195006
2,Men's / Unisex | Apparel | Google Merchandise ...,MensUnisex,164270
3,YouTube | Shop by Brand | Google Merchandise S...,YouTube,137918
4,Sale | Google Merchandise Store,Sale,131757
5,The Google Merchandise Store - Log In,TheGoogleMerchandiseStoreLogIn,123091
6,Checkout Your Information,CheckoutYourInformation,110781
7,Store search results,Storesearchresults,102228
8,Google Dino Game Tee,GoogleDinoGameTee,82167
9,Drinkware | Lifestyle | Google Merchandise Store,Drinkware,82156


In [34]:
def get_query_string(page_title_keys, page_title):
  """
  The function accepts:
    page_title_keys: All the keys from page_title (extracted from the previous function) that are used as column names and variable names.
    page_title: page_title dataframe that contains all columns

    This function achieves two things:
    1) Dynamically generates the WHEN matching query
    2) Dynamically generates the SELECT query

    case_when_string_list: Empty list that will store CASE WHEN dynamic SQL query string based on page_title keys (column names)
    select_key_string_list: Empty list that will store SELECT dynamic SQL query string based on page_title keys (column names)
  """
  case_when_string_list = []
  select_key_string_list = []

  for eachkey in page_title_keys:
    select_key_string = \
    f'IFNULL(item_page_view_table.pageVisit_count_sum_{eachkey},0) AS {eachkey}_visit_count,\n'
    select_key_string_list.append(select_key_string)

  for index, row in page_title.iterrows():
      case_when_string = \
      f"""WHEN ep.value.string_value LIKE "{row['page_title']}" THEN '{row['key']}' \n"""
      case_when_string_list.append(case_when_string)
  return (' '.join(select_key_string_list),
          ' '.join(case_when_string_list))
    

In [36]:
(select_key_string, case_when_string) = \
    get_query_string(list(df['key']),df)

In [28]:
print(tuple(df['key']))

('Apparel', 'Shopping Cart', 'Men s   Unisex', 'YouTube', 'Sale', 'The Google Merchandise Store   Log In', 'Checkout Your Information', 'Store search results', 'Google Dino Game Tee', 'Drinkware', 'New', 'Bags', 'Hats', 'Womens', 'Campus Collection', 'Page Unavailable', 'Eco Friendly', 'Payment Method')


In [37]:
print(select_key_string)

IFNULL(item_page_view_table.pageVisit_count_sum_Apparel,0) AS Apparel_visit_count,
 IFNULL(item_page_view_table.pageVisit_count_sum_ShoppingCart,0) AS ShoppingCart_visit_count,
 IFNULL(item_page_view_table.pageVisit_count_sum_MensUnisex,0) AS MensUnisex_visit_count,
 IFNULL(item_page_view_table.pageVisit_count_sum_YouTube,0) AS YouTube_visit_count,
 IFNULL(item_page_view_table.pageVisit_count_sum_Sale,0) AS Sale_visit_count,
 IFNULL(item_page_view_table.pageVisit_count_sum_TheGoogleMerchandiseStoreLogIn,0) AS TheGoogleMerchandiseStoreLogIn_visit_count,
 IFNULL(item_page_view_table.pageVisit_count_sum_CheckoutYourInformation,0) AS CheckoutYourInformation_visit_count,
 IFNULL(item_page_view_table.pageVisit_count_sum_Storesearchresults,0) AS Storesearchresults_visit_count,
 IFNULL(item_page_view_table.pageVisit_count_sum_GoogleDinoGameTee,0) AS GoogleDinoGameTee_visit_count,
 IFNULL(item_page_view_table.pageVisit_count_sum_Drinkware,0) AS Drinkware_visit_count,
 IFNULL(item_page_view_tabl

In [38]:
print(case_when_string)

WHEN ep.value.string_value LIKE "Apparel | Google Merchandise Store" THEN 'Apparel' 
 WHEN ep.value.string_value LIKE "Shopping Cart" THEN 'ShoppingCart' 
 WHEN ep.value.string_value LIKE "Men's / Unisex | Apparel | Google Merchandise Store" THEN 'MensUnisex' 
 WHEN ep.value.string_value LIKE "YouTube | Shop by Brand | Google Merchandise Store" THEN 'YouTube' 
 WHEN ep.value.string_value LIKE "Sale | Google Merchandise Store" THEN 'Sale' 
 WHEN ep.value.string_value LIKE "The Google Merchandise Store - Log In" THEN 'TheGoogleMerchandiseStoreLogIn' 
 WHEN ep.value.string_value LIKE "Checkout Your Information" THEN 'CheckoutYourInformation' 
 WHEN ep.value.string_value LIKE "Store search results" THEN 'Storesearchresults' 
 WHEN ep.value.string_value LIKE "Google Dino Game Tee" THEN 'GoogleDinoGameTee' 
 WHEN ep.value.string_value LIKE "Drinkware | Lifestyle | Google Merchandise Store" THEN 'Drinkware' 
 WHEN ep.value.string_value LIKE "New | Google Merchandise Store" THEN 'New' 
 WHEN e

In [40]:
query = f"""
CREATE OR REPLACE TABLE
  ga4_ecomm_feature_set.ga4_features AS
WITH 
  total_events_table AS (
    SELECT 
    user_pseudo_id,
    COUNT(DISTINCT(event_date)) AS total_event_count,
FROM 
  `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events_*`
WHERE
  event_date BETWEEN '{START_DATE}' AND '{END_DATE}'
GROUP BY 1),

add_to_cart_table AS (
  SELECT 
    user_pseudo_id,
    COUNT(*) AS add_to_cart_count
    FROM
        `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events_*`,
    UNNEST (event_params) AS eo
    WHERE 
      event_name = 'add_to_cart'
    AND
      event_date BETWEEN '{START_DATE}' AND '{END_DATE}'
    GROUP BY 1
),

purchase_table AS (
  SELECT 
    user_pseudo_id,
    COUNT(*) AS purchased_count
  FROM 
   `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events_*`,
    UNNEST (event_params) AS eo
    WHERE 
      event_name = 'purchase'
    AND
      event_date BETWEEN '{START_DATE}' AND '{END_DATE}'
    GROUP BY 1
),

browsing_features_table AS (
  SELECT
  user_pseudo_id,
  MAX(device.category) AS favorite_device_medium,
  MAX(device.mobile_brand_name) AS favorite_mobile_company_name,
  MAX(device.LANGUAGE) AS favorite_lang,
  MAX(geo.country) AS most_used_country,
  MAX(
    IF
    (events.key = "campaign",
    events.value.string_value, NULL)) AS most_used_campaign,
  ROUND(AVG(
    IF 
    (events.key = "engagement_time_msec",
    events.value.int_value, NULL)) / 60000,2) AS average_engagement_time_minute,
  FROM 
   `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events_*`,
    UNNEST(event_params) AS events,
    UNNEST(items) AS item
  WHERE
    event_date BETWEEN '{START_DATE}' AND '{END_DATE}'
  GROUP BY 1 ) 


  SELECT
  item_page_view_table.user_pseudo_id,
  {select_key_string}
  tet.total_event_count,
  IFNULL(atc.add_to_cart_count, 0) AS count_add_to_cart,
  IFNULL(pt.purchased_count,0) AS count_purchase,
  IFNULL(bft.favorite_device_medium,'NotAvailable') AS favorite_device_medium,
  IFNULL(bft.favorite_mobile_company_name, 'NotAvailable') AS  favorite_mobile_company_name,
  IFNULL(bft.favorite_lang, 'NotAvailable') AS favorite_language,
  IFNULL(bft.most_used_country,'NotAvailable') AS most_used_country,
  IFNULL(bft.most_used_campaign,'NotAvailable') AS most_used_campaign,
  IFNULL(bft.average_engagement_time_minute, 0) AS average_engagement_time_minute
  FROM (
    SELECT
    *
  FROM (
    SELECT 
      user_pseudo_id,
      CASE {case_when_string} ELSE "Others" END AS page_title,
      COUNT(ep.value.string_value) AS pageVisit_count
    FROM `{PROJECT_ID_DATA}.{DATASET_ID_DATA}.events_*`,
    UNNEST(event_params) AS ep
    WHERE
     ep.key = 'page_title'
     AND ep.value.string_value IN {tuple(df['page_title'])}
     AND event_date BETWEEN '{START_DATE}' AND '{END_DATE}'
     AND event_name = 'page_view'
    GROUP BY 1, 2)
    PIVOT 
    (SUM(pageVisit_count) AS pageVisit_count_sum FOR page_title IN {tuple(df['key'])})) item_page_view_table

  LEFT JOIN total_events_table tet ON tet.user_pseudo_id = item_page_view_table.user_pseudo_id
  LEFT JOIN add_to_cart_table atc ON atc.user_pseudo_id = item_page_view_table.user_pseudo_id
  LEFT JOIN purchase_table pt ON pt.user_pseudo_id = item_page_view_table.user_pseudo_id
  LEFT JOIN browsing_features_table bft ON bft.user_pseudo_id = item_page_view_table.user_pseudo_id

  WHERE
     item_page_view_table.user_pseudo_id IS NOT NULL
  """
query_job = client.query(query)
  
#print(query)



In [41]:
query = """
SELECT * FROM `ga4_ecomm_feature_set.ga4_features`
LIMIT 5
"""

query_job = client.query(query)
result_df = query_job.to_dataframe()
result_df

,user_pseudo_id,Apparel_visit_count,ShoppingCart_visit_count,MensUnisex_visit_count,YouTube_visit_count,Sale_visit_count,TheGoogleMerchandiseStoreLogIn_visit_count,CheckoutYourInformation_visit_count,Storesearchresults_visit_count,GoogleDinoGameTee_visit_count,Drinkware_visit_count,New_visit_count,Bags_visit_count,Hats_visit_count,Womens_visit_count,CampusCollection_visit_count,PageUnavailable_visit_count,EcoFriendly_visit_count,PaymentMethod_visit_count,total_event_count,count_add_to_cart,count_purchase,favorite_device_medium,favorite_mobile_company_name,favorite_language,most_used_country,most_used_campaign,average_engagement_time_minute
0,6975042.6074322973,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,1,0,0,desktop,Apple,en-us,United States,(referral),0.13
1,80435974.9462183494,0,4,1,0,0,2,2,0,0,0,1,0,1,11,0,0,1,3,2,120,20,desktop,Apple,es-es,United States,(referral),0.28
2,7920382.2659248129,0,1,1,0,1,0,0,1,0,0,1,0,0,0,1,0,2,0,5,0,0,desktop,Apple,en-us,United States,(referral),1.38
3,10626696.1834586147,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,0,mobile,Apple,NotAvailable,India,(referral),0.00
4,1183796.8486859683,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,desktop,Apple,en-us,Canada,(organic),0.00
